In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from tensorflow.keras import regularizers
from keras import backend as K
import tensorflow as tf

class SmallVGGNet:
	@staticmethod
	def build(width, height, depth, classes):

		model = Sequential()

		inputShape = (height, width, depth)
		chanDim = -1

		# (CONV => RELU) * 2 => POOL layer set
		model.add(BatchNormalization(input_shape=inputShape, axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))				#, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.2))

		# (CONV => RELU) * 2 => POOL layer set
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))				#	, kernel_regularizer=tf.keras.regularizers.l2(l=0.01) 
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))	
		model.add(Conv2D(64, (3, 3), padding="same"))				#, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
		model.add(Activation("relu"))
		model.add(Dropout(0.25))
		model.add(GlobalAveragePooling2D(data_format='channels_last'))

		model.add(BatchNormalization(axis=chanDim))
		model.add(Dense(256))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))
		
		model.add(Dense(16))		
		model.add(Activation("relu"))

		model.add(Dense(classes))
		model.add(Activation("relu"))
		# return the constructed network architecture
		return model

In [0]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
import sys
matplotlib.use("Agg")
# import the necessary packages
sys.path.insert(1, "/gdrive/My Drive/App")
import sys
import skimage
from skimage import img_as_float
from numpy import genfromtxt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, Model, callbacks
from keras.optimizers import SGD, RMSprop, Adam
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
from keras.applications import ResNet50
import pickle
import cv2
import keras.models 
import os

INIT_LR = 3e-3
EPOCHS = 50
BS = 32
SIZE = 64
SIZECUT = 32

# initialize the data and labels
def load_attributes(inputPath):
  cols = ["ringing", "defocus","noise"]
  col = ["noise"]
  col2 =["ringing"]
  df = pd.read_csv(inputPath, sep=",",  usecols=col)
  return df

def process_image(im):
	w = im[im.shape[0]//2 - SIZECUT:im.shape[0]//2 + SIZECUT, im.shape[1]//2 - SIZECUT:im.shape[1]//2 + SIZECUT]
	return w

print("[INFO] loading images...")
data = []
attrib = []
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("/gdrive/My Drive/App/database")))
i = 0
# loop over the input images
for imagePath in imagePaths:
  if i == 5500:
    break
  image = cv2.imread(imagePath)  
  img = process_image(image)
  data.append(img) 

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype='float32')/ 255

print("[INFO] loading attributes...")
attrib = load_attributes("/gdrive/My Drive/App/Param.csv")
attrib = attrib[:5500]

print("[INFO] split the data...")
(trainX, testX, trainY, testY) = train_test_split(data,
	attrib, test_size=0.25, random_state=42)

optA = Adam(lr=INIT_LR)

model = SmallVGGNet.build(width=SIZE, height=SIZE, depth=3, classes=1)
model.compile(loss="mse", optimizer=optA,
	metrics=["accuracy"])
'''
filepath = "/gdrive/My Drive/App/Model.h5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
'''
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=BS)   #, callbacks=callbacks_list

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
#plt.plot(N, H.history["accuracy"], label="train_acc")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.savefig("/gdrive/My Drive/App/N/Noise_train.png")

from keras.utils import plot_model
plot_model(model, to_file='/gdrive/My Drive/App/N/CnnN.png', show_shapes=True)

# evaluate the network
print("[INFO] evaluating network...")
result = model.evaluate(testX, testY, batch_size=BS)

print('test loss, test acc:', result)
predictions = model.predict(testX, batch_size=BS)

print("[INFO] serializing network...")
model.save("/gdrive/My Drive/App/N/CnnN_last.h5")

print("MSE  (testY, predictions)")
print(mean_squared_error(testY, predictions, squared=False))

'''
print(predictions[:10])
print(testY[:10])
predictions2 = model.predict(trainX[:10], batch_size=BS)
print(predictions2[:10], end='\t')
print(trainY[:10])
'''


[INFO] loading images...
